# 透過 bt.indicator 建立客製化指標

In [ ]:
import backtrader as bt
import backtrader.indicators as btind

# 1. 繼承 bt.Indicator
class Simple_EMA_Indicator(bt.Indicator):
    
    # 2. 定義參數 (讓使用者可以設定 EMA 的週期)
    params = dict(
        period=10,  # 預設週期為 10 天
    )

    # 3. 定義輸出線
    # 我們的指標只輸出一個結果，命名為 'ema_output'
    lines = ('ema_output',) 

    # 讓它在策略圖中單獨顯示 (可選)
    plotinfo = dict(
        subplot=False,
        ) 

    def __init__(self):
        
        # 3.1 嵌入預設的 EMA 指標
        # 我們使用 btind.ExponentialMovingAverage (EMA)
        # 輸入資料線是 self.data (即傳給這個客製化指標的資料)
        # 週期使用我們定義的參數 self.p.period
        self.ema_calc = btind.ExponentialMovingAverage(
            self.data.close,  # 通常 EMA 是計算收盤價
            period=self.p.period
        ) 

    # 4. 實作計算邏輯
    # 實際上可以把 這段 next 內的邏輯寫在 __init__ 中。
    # 實務上只有需要計算較複雜的邏輯（無法用線性運算）時，才會實作 next() 方法。 盡量多使用線性運算效能會比較好。
    def next(self):
        
        # 4.1 將內嵌指標 (self.ema_calc) 的當前值，賦值給客製化指標的輸出線 (self.lines.ema_output)
        
        # EMA 指標的輸出線就是它的本身 (索引 0 或 .ema)
        # self.ema_calc[0] 代表今天計算出來的 10 日 EMA 值
        
        self.lines.ema_output[0] = self.ema_calc[0]
        
        # 由於這個客製化指標只是一個簡單的包裝，next 邏輯非常簡單
        pass

# --- 策略中使用範例 ---
class MySimpleEMATrade(bt.Strategy):
    def __init__(self):
        # 將客製化指標加入策略
        # 這裡設定 EMA 週期為 7 天
        self.custom_ema = Simple_EMA_Indicator(self.data, period=7)
        
        # 也可以直接存取預設的 20 日 EMA 來比較
        self.default_ema = btind.EMA(self.data, period=20)
        
    def next(self):
        # 存取客製化指標的當前值
        custom_ema_value = self.custom_ema.ema_output[0]
        
        # 存取預設 EMA 的當前值
        default_ema_value = self.default_ema[0]
        
        current_close = self.data.close[0]
        
        # 簡單的買入邏輯：當價格高於 7 日 EMA (客製化指標) 且 7 日 EMA 高於 20 日 EMA 時買入
        if current_close > custom_ema_value and custom_ema_value > default_ema_value and not self.position:
            print(f"{self.data.datetime.date(0)} 🚀 買進! Close:{current_close:.2f}, Custom EMA(7):{custom_ema_value:.2f}")
            self.buy()

In [ ]:
# 線運算範例
import backtrader as bt

class DailyRangeIndicator(bt.Indicator):
    
    # 參數 (這裡不需要參數，因為只計算當日價差)
    params = ()  

    # 輸出線：輸出每日的高低價差
    lines = ('daily_range',) 
    
    # 讓它在策略圖中單獨顯示 (subplot=True 較為合適)
    plotinfo = dict(subplot=True) 

    def __init__(self):
        
        # 1. 取得 High 價資料線和 Low 價資料線
        high_line = self.data.high 
        low_line = self.data.low
        
        # 2. 核心：直接使用線運算 (Line Arithmetic) 進行相減
        # 將 high_line 減去 low_line 的結果，直接賦值給輸出線 self.lines.daily_range
        # Backtrader 會自動計算每一個交易日的 (High - Low)
        self.lines.daily_range = high_line - low_line
        
        # 由於所有的計算都在 __init__ 中定義，不需要實作 next() 方法。
        
# --- 策略：使用每日高低價差指標 ---
class RangeStrategy(bt.Strategy):
    
    def __init__(self):
        self.order = None
        
        # 實例化客製化指標
        self.daily_range = DailyRangeIndicator(self.data)
        
    def next(self):
        if self.order:
            return

        # 取得客製化指標的當前值 (今天的高低價差)
        current_range = self.daily_range.daily_range[0]
        current_close = self.data.close[0]

        # 簡單邏輯：如果價差大於 10 點，且沒有持倉，認為波動大，先觀望
        if current_range > 10.0 and not self.position:
            # print(f"{self.data.datetime.date(0)} 📈 波動過大 ({current_range:.2f})，等待入場...")
            pass
        
        # 簡單邏輯：如果價差小於 1 點，認為極度低迷，嘗試買入
        elif current_range < 1.0 and not self.position:
            self.order = self.buy()
            print(f"{self.data.datetime.date(0)} 🚀 買進! Close: {current_close:.2f}, Range: {current_range:.2f} (波動極小)")

    def notify_order(self, order):
        if order.status in [order.Completed, order.Canceled, order.Rejected]:
            self.order = None

In [ ]:
# 另一個範例
import backtrader as bt

# 繼承 bt.Indicator 建立客製化指標
class RateOfChange(bt.Indicator):
    
    # 1. 定義參數：ROC 需要一個週期 N
    params = dict(
        period=12,  # 預設週期為 12
    )

    # 2. 定義輸出線：指標輸出結果命名為 'roc'
    lines = ('roc',)
    
    # 讓它在單獨的副圖區顯示
    plotinfo = dict(subplot=True)

    # 3. 實作 Line Arithmetic
    def __init__(self):
        
        # 存取輸入資料的收盤價線
        dataclose = self.data.close
        
        # 核心步驟 A：創建一條延遲的 Line 物件
        # close_n_ago 上的今天的值，等於 dataclose 上的 N 天前的值
        # N = self.params.period
        close_n_ago = dataclose(-self.params.period)
        
        # 核心步驟 B：使用 Line Arithmetic 運算公式
        # self.lines.roc 是輸出行。Backtrader 會自動計算每一個時間點的 ROC
        # (今日收盤價 - N天前收盤價) / N天前收盤價 * 100
        self.lines.roc = ((dataclose - close_n_ago) / close_n_ago) * 100.0
        
        # 確保呼叫父類別的 __init__ 方法，這對於 Backtrader 內部設置和繪圖非常重要
        # 策略類別卻不需要
        super(RateOfChange, self).__init__()
        
        # ⭐️ 注意：由於所有計算都已在 __init__ 中以線運算定義，
        # 策略或指標不需要 next() 函式，程式碼更簡潔、執行效率更高。